In [1]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import csv
%matplotlib inline

#Preprocesing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
data = pd.read_csv('data.csv')
data.head()

,filename,chroma_stft,chroma_stft_var,rmse,rmse_var,spectral_centroid,spectral_centroid_var,spectral_bandwidth,spectral_bandwidth_var,rolloff,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00000.wav,0.349943,0.088760,0.130225,0.002829,1784.420446,129789.689238,2002.650192,85897.037189,3806.485316,...,8.810669,-3.667368,5.751691,-5.162763,0.750948,-1.691938,-0.409953,-2.300209,1.219929,blues
1,blues.00001.wav,0.340983,0.094981,0.095918,0.002373,1529.835316,375984.023922,2038.617579,213794.462833,3548.820207,...,5.376803,-2.239120,4.216963,-6.012273,0.936109,-0.716537,0.293876,-0.287431,0.531573,blues
2,blues.00002.wav,0.363603,0.085287,0.175573,0.002748,1552.481958,156526.317715,1747.165985,76046.707240,3040.514948,...,5.789265,-8.905224,-1.083720,-9.218359,2.455806,-7.726901,-1.815723,-3.433434,-2.226821,blues
3,blues.00003.wav,0.404779,0.094009,0.141191,0.006339,1070.119953,184498.418348,1596.333948,166561.564811,2185.028454,...,6.087677,-2.476421,-1.073890,-2.874778,0.780977,-3.316932,0.637982,-0.619690,-3.408233,blues
4,blues.00004.wav,0.308590,0.087850,0.091563,0.002304,1835.494603,343350.842007,1748.362448,88466.013315,3580.945013,...,-2.806384,-6.934123,-7.558618,-9.173553,-4.512165,-5.453538,-0.924161,-4.409333,-11.703781,blues


In [3]:
data = data.drop(['filename'],axis=1)
data.head()

,chroma_stft,chroma_stft_var,rmse,rmse_var,spectral_centroid,spectral_centroid_var,spectral_bandwidth,spectral_bandwidth_var,rolloff,rolloff_var,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,0.349943,0.088760,0.130225,0.002829,1784.420446,129789.689238,2002.650192,85897.037189,3806.485316,9.016619e+05,...,8.810669,-3.667368,5.751691,-5.162763,0.750948,-1.691938,-0.409953,-2.300209,1.219929,blues
1,0.340983,0.094981,0.095918,0.002373,1529.835316,375984.023922,2038.617579,213794.462833,3548.820207,2.976486e+06,...,5.376803,-2.239120,4.216963,-6.012273,0.936109,-0.716537,0.293876,-0.287431,0.531573,blues
2,0.363603,0.085287,0.175573,0.002748,1552.481958,156526.317715,1747.165985,76046.707240,3040.514948,7.831159e+05,...,5.789265,-8.905224,-1.083720,-9.218359,2.455806,-7.726901,-1.815723,-3.433434,-2.226821,blues
3,0.404779,0.094009,0.141191,0.006339,1070.119953,184498.418348,1596.333948,166561.564811,2185.028454,1.494250e+06,...,6.087677,-2.476421,-1.073890,-2.874778,0.780977,-3.316932,0.637982,-0.619690,-3.408233,blues
4,0.308590,0.087850,0.091563,0.002304,1835.494603,343350.842007,1748.362448,88466.013315,3580.945013,1.572304e+06,...,-2.806384,-6.934123,-7.558618,-9.173553,-4.512165,-5.453538,-0.924161,-4.409333,-11.703781,blues


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 33 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   chroma_stft             1000 non-null   float64
 1   chroma_stft_var         1000 non-null   float64
 2   rmse                    1000 non-null   float64
 3   rmse_var                1000 non-null   float64
 4   spectral_centroid       1000 non-null   float64
 5   spectral_centroid_var   1000 non-null   float64
 6   spectral_bandwidth      1000 non-null   float64
 7   spectral_bandwidth_var  1000 non-null   float64
 8   rolloff                 1000 non-null   float64
 9   rolloff_var             1000 non-null   float64
 10  zero_crossing_rate      1000 non-null   float64
 11  zero_crossing_rate_var  1000 non-null   float64
 12  mfcc1                   1000 non-null   float64
 13  mfcc2                   1000 non-null   float64
 14  mfcc3                   1000 non-null   f

In [5]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)
print(y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 

In [6]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

In [7]:
X

array([[-0.35174835,  0.31266712, -0.01072298, ..., -0.23719143,
         0.00761125,  0.60349827],
       [-0.46146555,  1.11697192, -0.53326615, ..., -0.05518971,
         0.54382356,  0.42403527],
       [-0.18448354, -0.13630778,  0.68001209, ..., -0.60070696,
        -0.29428465, -0.29511276],
       ...,
       [ 0.6543177 , -1.42879075, -0.75110651, ...,  0.76028073,
        -2.73474442, -0.2638744 ],
       [-0.19983743,  0.66809238, -0.71651358, ...,  0.27176634,
        -0.72311177, -0.64936233],
       [-0.25070237, -0.06394035, -1.16473892, ..., -0.1250688 ,
         0.08171812,  0.58748954]])

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40,random_state=100)

In [9]:
X_test

array([[-0.67390123,  0.37589976,  0.19365927, ..., -1.18306136,
         0.10144072, -1.22311633],
       [ 1.09095382, -0.73891826,  0.6351997 , ...,  0.47548657,
         0.41068117,  0.13050736],
       [-1.14779642,  0.76220451, -0.37001267, ...,  0.09204737,
         1.02494999,  0.22427372],
       ...,
       [ 1.43520056, -1.74076511, -0.20701657, ...,  0.86905096,
        -0.696635  ,  1.13514021],
       [ 1.14196874,  1.08469485,  0.91412817, ...,  0.18376748,
         0.49432222, -0.24433972],
       [-1.68939026, -0.35469301, -1.5613929 , ...,  0.05097891,
         0.89457775, -0.51284525]])

In [10]:
from sklearn.svm import SVC
model=SVC()
model.fit(X_train,y_train)

SVC()

In [11]:
prediction = model.predict(X_test)
type(prediction)

numpy.ndarray

In [12]:
prediction[170]

4

In [13]:
from sklearn.metrics import classification_report,confusion_matrix

In [14]:
print(classification_report(y_test,prediction))

              precision    recall  f1-score   support

           0       0.67      0.70      0.68        37
           1       0.94      0.88      0.91        52
           2       0.62      0.49      0.55        47
           3       0.36      0.71      0.48        34
           4       0.69      0.51      0.59        43
           5       0.76      0.71      0.74        45
           6       0.86      0.78      0.82        41
           7       0.75      0.66      0.70        32
           8       0.51      0.59      0.55        37
           9       0.54      0.44      0.48        32

    accuracy                           0.66       400
   macro avg       0.67      0.65      0.65       400
weighted avg       0.69      0.66      0.66       400



In [15]:
print(confusion_matrix(y_test,prediction))

[[26  0  1  2  1  2  3  0  0  2]
 [ 0 46  1  1  0  1  0  0  1  2]
 [ 4  0 23  8  0  6  0  1  2  3]
 [ 0  0  1 24  4  0  1  1  1  2]
 [ 1  0  1  8 22  0  0  3  8  0]
 [ 3  3  4  0  0 32  0  0  2  1]
 [ 4  0  1  3  0  0 32  0  1  0]
 [ 0  0  2  6  1  0  0 21  2  0]
 [ 0  0  2  5  4  1  0  1 22  2]
 [ 1  0  1 10  0  0  1  1  4 14]]


In [24]:
input_file = 'genres\\jazz\\jazz.00005.wav'

In [25]:
y, sr = librosa.load(input_file, mono=True, duration=30)
chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
rmse = librosa.feature.rms(y=y)
spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
zcr = librosa.feature.zero_crossing_rate(y)
mfcc = librosa.feature.mfcc(y=y, sr=sr)

arr = np.array([[np.mean(chroma_stft), np.var(chroma_stft) ,np.mean(rmse) ,np.var(rmse) ,np.mean(spec_cent) ,np.var(spec_cent) ,np.mean(spec_bw) ,np.var(spec_bw) ,np.mean(rolloff) ,np.var(rolloff) ,np.mean(zcr) ,np.var(zcr)]])
for e in mfcc:
    arr=np.append(arr,np.mean(e));
ar=np.array([arr])  

In [26]:
arr.shape

(32,)

In [27]:
type(ar)

numpy.ndarray

In [28]:
len(ar[0])

32

In [29]:
a=scaler.transform(np.array(ar,dtype=float))
a

array([[-1.59716875,  1.30264762, -0.35602686, -0.22416626, -1.30487486,
        -0.4570667 , -1.32349997,  1.03260952, -1.32498334,  0.01331939,
        -1.09476314, -0.68781869, -1.01624082,  0.65909162,  0.36872253,
        -0.33628298,  0.11005669,  0.71929203,  0.56044739, -0.09130187,
         0.77518841, -0.25495579,  0.33311432, -0.03598123,  0.73259021,
         1.13174363,  0.94262327,  1.29580905,  1.06214354,  0.115344  ,
         1.08312728, -0.06533899]])

In [30]:
pred=model.predict(a)

In [31]:
if(pred==0):
    print("Blue")
elif(pred==1):
    print("Classical")
elif(pred==2):
    print("Country")
elif(pred==3):
    print("Disco")
elif(pred==4):
    print("Hiphop")
elif(pred==5):
    print("Jazz")
elif(pred==6):
    print("Metal")
elif(pred==7):
    print("Pop")
elif(pred==8):
    print("Reggae")
elif(pred==9):
    print("Rock")


Jazz
